---

## Задача поиска схожих по смыслу предложений

Задача кода в даном ноутбуке -
построить ранжирующую функцию $a$ : $X \rightarrow R$ вопросов [StackOverflow](https://stackoverflow.com) на основе семантического векторного представления

## Математическая формулировка задачи ранжировния (Learning to Rank)

* $X$ - множество объектов
* $X^l = \{x_1, x_2, ..., x_l\}$ - обучающая выборка
<br>На обучающей выборке задан порядок между некоторыми элементами, то есть нам известно, что некий объект выборки более релевантный для нас, чем другой:
* $i \prec j$ - порядок пары индексов объектов на выборке $X^l$ c индексами $i$ и $j$
### Задача:
построить ранжирующую функцию $a$ : $X \rightarrow R$ такую, что
$$i \prec j \Rightarrow a(x_i) < a(x_j)$$

### Embeddings

Будем использовать предобученные векторные представления слов на постах Stack Overflow.<br>
[A word2vec model trained on Stack Overflow posts](https://github.com/vefstathiou/SO_word2vec)

In [ ]:
!wget https://zenodo.org/record/1199620/files/SO_vectors_200.bin?download=1

--2024-03-04 18:08:16--  https://zenodo.org/record/1199620/files/SO_vectors_200.bin?download=1
Resolving zenodo.org (zenodo.org)... 188.185.79.172, 188.184.103.159, 188.184.98.238, ...
Connecting to zenodo.org (zenodo.org)|188.185.79.172|:443... connected.
HTTP request sent, awaiting response... 301 MOVED PERMANENTLY
Location: /records/1199620/files/SO_vectors_200.bin [following]
--2024-03-04 18:08:16--  https://zenodo.org/records/1199620/files/SO_vectors_200.bin
Reusing existing connection to zenodo.org:443.
HTTP request sent, awaiting response... 200 OK
Length: 1453905423 (1.4G) [application/octet-stream]
Saving to: 'SO_vectors_200.bin?download=1.1'

SO_vectors_200.bin? 100%[===================>]   1.35G  29.6MB/s    in 48s     

2024-03-04 18:09:05 (28.8 MB/s) - 'SO_vectors_200.bin?download=1.1' saved [1453905423/1453905423]



In [ ]:
from gensim.models.keyedvectors import KeyedVectors

# load_word2vec_format используется для загрузки предобученных векторов слов в формате Word2Vec.
wv_embeddings = KeyedVectors.load_word2vec_format("SO_vectors_200.bin?download=1", binary=True)

Посмотрим на примере одного слова, что из себя представляет embedding

In [ ]:
word = 'dog'
if word in wv_embeddings:
    print(wv_embeddings[word].dtype, wv_embeddings[word].shape)
    print (wv_embeddings[word])

float32 (200,)
[ 0.6851772  -1.2778991  -0.41913974  1.3623164  -3.1675398   0.09950767
  0.6402681  -1.1245339  -0.6699619  -0.6998852   0.4936771  -0.40500194
 -3.0706816  -2.2809966   0.85798043  2.7093108   0.3492745  -0.03494101
 -0.22330493  1.2290467   1.7755157  -3.158358   -0.6515983   0.7224096
  2.3193083  -1.7969862   0.40903398 -2.744604   -1.7179952  -0.914309
 -0.75887376 -0.35140672 -0.5182776  -1.9097351  -0.8300773   0.02147918
  1.1783471   0.03169126 -0.3069023   1.6666299   0.6711357  -2.1706133
 -0.11800487  0.22336982 -1.2075394  -0.86297905 -0.63865614  1.1733794
  0.10022762  0.7017279   2.7290728  -0.4640484  -2.1719306  -0.3562852
 -1.8449957   0.10270727  1.1125596  -0.8364318   1.9513408  -0.97937447
  1.2650859   0.06809282  0.6477318  -0.52431005 -0.6103959  -2.979829
 -0.7889965  -0.11004248  1.7603841  -1.0547444  -0.98998    -1.1834062
 -2.5359967  -0.35286787  0.7733574   0.16300043  0.76991326  1.9223119
 -1.2843752  -1.5023688   0.15765315 -2.415087

In [ ]:
print(f"Num of words: {len(wv_embeddings.index_to_key)}")

Num of words: 1787145


Чтобы ответить на первый вопрос (входит ли слов `cat` топ-5 близких слов к слову `dog`), Найдем наиболее близкие слова к слову `dog`:

In [ ]:
# method most_simmilar

if word in wv_embeddings:
    similar_words = wv_embeddings.most_similar(positive=word, topn=5)
    print("топ-5 близких слов к слову dog:")
    for i, (word, similarity) in enumerate(similar_words):
        print(f"{i+1}. {word}: {similarity}")

топ-5 близких слов к слову dog:
1. animal: 0.8564180135726929
2. dogs: 0.7880866527557373
3. mammal: 0.7623804211616516
4. cats: 0.7621253728866577
5. animals: 0.760793924331665


**Таким образом, слово cat не входит, но входит cats. Место 4**

### Векторные представления текста

Перейдем от векторных представлений отдельных слов к векторным представлениям вопросов, как к **среднему** векторов всех слов в вопросе. Если для какого-то слова нет предобученного вектора, то его нужно пропустить. Если вопрос не содержит ни одного известного слова, то нужно вернуть нулевой вектор.

In [ ]:
pip install spacy

Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install pymorphy2

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import numpy as np
import re

# Токенайзер, возвращающий слова без знаков пунктуации без применения нормализации
class MyTokenizer:
    def __init__(self):
        pass
    def tokenize(self, text):
        return re.findall('\w+', text)
tokenizer = MyTokenizer()

In [ ]:
# pymorphy2 возвращает слова,пунктуацию, применяет нормализацию с помощью pymorphy2
import pymorphy2
morph = pymorphy2.MorphAnalyzer()
class Pymorphy2Tokenizer:
    def __init__(self):
        pass
    def tokenize(self, text):
        pymorphy_results = list(map(lambda x: morph.parse(x), text.split(' ')))
        return [res[0].normal_form for res in pymorphy_results]

pymorphy2_tokenizer = Pymorphy2Tokenizer()

In [ ]:
raw = """DENNIS: Listen, strange women lying in ponds distributing swords is no basis for a system of government.  Supreme executive power derives from a mandate from the masses, not from some farcical aquatic ceremony."""

raw_ru = """Не существует научных доказательств в пользу эффективности НЛП, оно признано псевдонаукой. Систематические обзоры указывают, что НЛП основано на устаревших представлениях об устройстве мозга, несовместимо с современной неврологией и содержит ряд фактических ошибок."""

In [ ]:
print(tokenizer.tokenize(raw),tokenizer.tokenize(raw_ru)  )

['DENNIS', 'Listen', 'strange', 'women', 'lying', 'in', 'ponds', 'distributing', 'swords', 'is', 'no', 'basis', 'for', 'a', 'system', 'of', 'government', 'Supreme', 'executive', 'power', 'derives', 'from', 'a', 'mandate', 'from', 'the', 'masses', 'not', 'from', 'some', 'farcical', 'aquatic', 'ceremony'] ['Не', 'существует', 'научных', 'доказательств', 'в', 'пользу', 'эффективности', 'НЛП', 'оно', 'признано', 'псевдонаукой', 'Систематические', 'обзоры', 'указывают', 'что', 'НЛП', 'основано', 'на', 'устаревших', 'представлениях', 'об', 'устройстве', 'мозга', 'несовместимо', 'с', 'современной', 'неврологией', 'и', 'содержит', 'ряд', 'фактических', 'ошибок']


In [ ]:
# print(spacy_tokenizer.tokenize(raw),spacy_tokenizer.tokenize(raw_ru)  )

In [ ]:
print(pymorphy2_tokenizer.tokenize(raw),pymorphy2_tokenizer.tokenize(raw_ru)  )

['dennis:', 'listen,', 'strange', 'women', 'lying', 'in', 'ponds', 'distributing', 'swords', 'is', 'no', 'basis', 'for', 'a', 'system', 'of', 'government.', '', 'supreme', 'executive', 'power', 'derives', 'from', 'a', 'mandate', 'from', 'the', 'masses,', 'not', 'from', 'some', 'farcical', 'aquatic', 'ceremony.'] ['не', 'существовать', 'научный', 'доказательство', 'в', 'польза', 'эффективность', 'нлп,', 'оно', 'признать', 'псевдонаукой.', 'систематический', 'обзор', 'указывают,', 'что', 'нлп', 'основать', 'на', 'устаревший', 'представление', 'о', 'устройство', 'мозга,', 'несовместимый', 'с', 'современный', 'неврология', 'и', 'содержать', 'ряд', 'фактический', 'ошибок.']


In [ ]:
def question_to_vec(question, embeddings, tokenizer, dim=200):
    """
        question: строка
        embeddings: наше векторное представление
        dim: размер любого вектора в нашем представлении

        return: векторное представление для вопроса
    """

    # sum_token=[]
    sum_token = np.zeros(dim)
    tokenized_question = tokenizer.tokenize(question)  # Токенизируем вопрос

    for word in tokenized_question:
        if word in embeddings:
            sum_token += embeddings[word]  # Добавляем вектор слова к сумме
        else:
            continue  # Пропускаем слова, которых нет в embeddings

    return sum_token/len(tokenized_question)

Теперь у нас есть метод для создания векторного представления любого предложения.

#### ответ на вопрос 2 (Какая третья(с индексом 2) компонента вектора предложения `I love neural networks` (округлите до 2 знаков после запятой)):

In [ ]:
round (question_to_vec('I love neural networks', wv_embeddings, tokenizer )[2], 2)

-0.96

### Оценка близости текстов

Представим, что мы используем идеальные векторные представления слов. Тогда косинусное расстояние между дублирующими предложениями должно быть меньше, чем между случайно взятыми предложениями.

Сгенерируем для каждого из $N$ вопросов $R$ случайных отрицательных примеров и примешаем к ним также настоящие дубликаты. Для каждого вопроса будем ранжировать с помощью нашей модели $R + 1$ примеров и смотреть на позицию дубликата. Мы хотим, чтобы дубликат был первым в ранжированном списке.

#### Hits@K
Первой простой метрикой будет количество корректных попаданий для какого-то $K$:
$$ \text{Hits@K} = \frac{1}{N}\sum_{i=1}^N \, [rank\_q_i^{'} \le K],$$
* $\begin{equation*}
[x < 0 ] \equiv
 \begin{cases}
   1, &x < 0\\
   0, &x \geq 0
 \end{cases}
\end{equation*}$ - индикаторная функция
* $q_i$ - $i$-ый вопрос
* $q_i^{'}$ - его дубликат
* $rank\_q_i^{'}$ - позиция дубликата в ранжированном списке ближайших предложений для вопроса $q_i$.

#### DCG@K
Второй метрикой будет упрощенная DCG метрика, учитывающая порядок элементов в списке путем домножения релевантности элемента на вес равный обратному логарифму номера позиции::
$$ \text{DCG@K} = \frac{1}{N} \sum_{i=1}^N\frac{1}{\log_2(1+rank\_q_i^{'})}\cdot[rank\_q_i^{'} \le K],$$
С такой метрикой модель штрафуется за большой ранк корректного ответа

#### Ответ на вопрос 3 (максимум `Hits@47 - DCG@1`):

$$ \text{Hits@47}- \text{DCG@1}  = \frac{1}{N}\sum_{i=1}^N \, ([rank\_q_i^{'} \le 47]- \frac{1}{\log_2(1+rank\_q_i^{'})}\cdot[rank\_q_i^{'} \le 1]) ,$$

Максимум будет достигаться, когда $$ [rank\_q_i^{'} \le 47]- \frac{1}{\log_2(1+rank\_q_i^{'})}\cdot[rank\_q_i^{'} \le 1] = max ,$$


при любых значениях $$ rank\_q_i  $$ больших единицы $$ [rank\_q_i^{'} \le 1] $$ принимает значение 0 (а не 1), то есть второе слагаемое равно 0.

Наибольшее значение выражения равно 1




#### Пример оценок

Вычислим описанные выше метрики для игрушечного примера.
Пусть
* $N = 1$, $R = 3$
* <font color='green'>"Что такое python?"</font> - вопрос $q_1$
* <font color='red'>"Что такое язык python?"</font> - его дубликат $q_i^{'}$

Пусть модель выдала следующий ранжированный список кандидатов:

1. "Как изучить с++?"
2. <font color='red'>"Что такое язык python?"</font>
3. "Хочу учить Java"
4. "Не понимаю Tensorflow"

$\Rightarrow rank\_q_i^{'} = 2$

Вычислим метрику *Hits@K* для *K = 1, 4*:

- [K = 1] $\text{Hits@1} =  [rank\_q_i^{'} \le 1)] = 0$
- [K = 4] $\text{Hits@4} =  [rank\_q_i^{'} \le 4] = 1$

Вычислим метрику *DCG@K* для *K = 1, 4*:
- [K = 1] $\text{DCG@1} = \frac{1}{\log_2(1+2)}\cdot[2 \le 1] = 0$
- [K = 4] $\text{DCG@4} = \frac{1}{\log_2(1+2)}\cdot[2 \le 4] = \frac{1}{\log_2{3}}$

#### Ответ на вопрос 4 (вычислите `DCG@10`, если $rank\_q_i^{'} = 9$ (округлите до одного знака после запятой) ):

**DCG@10 = 0.3**

### HITS\_COUNT и DCG\_SCORE

Каждая функция имеет два аргумента: $dup\_ranks$ и $k$. $dup\_ranks$ является списком, который содержит рейтинги дубликатов(их позиции в ранжированном списке). Например, $dup\_ranks = [2]$ для примера, описанного выше.

In [ ]:
# def hits_count(dup_ranks, k):
#   if dup_ranks <= k :
#     hits_value = 1
#   else:
#     hits_value = 0
#   return hits_value

def hits_count(dup_ranks, k):
    """
        dup_ranks: list индексов дубликатов
        result: вернуть  Hits@k
    """
    hits = sum([np.zeros(1)[0] if rank > k else np.ones(1)[0] for rank in dup_ranks])
    # hits_value = round(sum([np.zeros(1)[0] if rank > k else np.ones(1)[0] for rank in dup_ranks]) / len(dup_ranks),3)
    return hits/len(dup_ranks)

# def dcg_score(dup_ranks, k):
#   if dup_ranks<=k:
#     dcg_value = 1/np.log2(1+dup_ranks)
#   else:
#     dcg_value= 0
#   return dcg_value

def dcg_score(dup_ranks, k):
    """
        dup_ranks: list индексов дубликатов
        result: вернуть DCG@k
    """
    dcg = sum([np.zeros(1)[0] if rank > k else 1/np.log2(1 + rank) for rank in dup_ranks])

    return dcg/len(dup_ranks)


Протестируем функции. Пусть $N = 1$, то есть один эксперимент. Будем искать копию вопроса и оценивать метрики.

In [ ]:
import pandas as pd

In [ ]:
copy_answers = ["How does the catch keyword determine the type of exception that was thrown",]

# наши кандидаты
candidates_ranking = [["How Can I Make These Links Rotate in PHP",
                       "How does the catch keyword determine the type of exception that was thrown",
                       "NSLog array description not memory address",
                       "PECL_HTTP not recognised php ubuntu"],]
# dup_ranks — позиции наших копий, так как эксперимент один, то этот массив длины 1
# dup_ranks = [candidates_ranking[i].index(copy_answers[i]) + 1 for i in range(len(copy_answers))][0]
dup_ranks =[2]


# вычисляем метрику для разных k
print('Ваш ответ HIT:', [hits_count(dup_ranks, k) for k in range(1, 5)])
print('Ваш ответ DCG:', [round(dcg_score(dup_ranks, k), 5) for k in range(1, 5)])

Ваш ответ HIT: [0.0, 1.0, 1.0, 1.0]
Ваш ответ DCG: [0.0, 0.63093, 0.63093, 0.63093]


должно получиться

In [ ]:
# correct_answers - метрика для разных k
correct_answers = pd.DataFrame([[0, 1, 1, 1], [0, 1 / (np.log2(3)), 1 / (np.log2(3)), 1 / (np.log2(3))]],
                               index=['HITS', 'DCG'], columns=range(1,5))
correct_answers

,1,2,3,4
HITS,0,1.00000,1.00000,1.00000
DCG,0,0.63093,0.63093,0.63093


### Данные
[arxiv link](https://drive.google.com/file/d/1QqT4D0EoqJTy7v9VrNCYD-m964XZFR7_/edit)

`train.tsv` - выборка для обучения.<br> В каждой строке через табуляцию записаны: **<вопрос>, <похожий вопрос>**

`validation.tsv` - тестовая выборка.<br> В каждой строке через табуляцию записаны: **<вопрос>, <похожий вопрос>, <отрицательный пример 1>, <отрицательный пример 2>, ...**

In [ ]:
# !unzip stackoverflow_similar_questions.zip

In [ ]:
def read_corpus(filename):
    data = []
    for line in open(filename, encoding='utf-8'):
        data.append(line.split('\t'))
    return data

Понадобится только файл validation.

In [ ]:
validation_data = read_corpus('/kaggle/input/trainvalid/validation.tsv')

Кол-во строк

In [ ]:
len(validation_data)

3760

Размер нескольких первых строк

In [ ]:
for i in range(5):
    print(i + 1, len(validation_data[i]))

1 1001
2 1001
3 1001
4 1001
5 1001


### Ранжирование без обучения

Реализуем функцию ранжирования кандидатов на основе косинусного расстояния. Функция должна по списку кандидатов вернуть отсортированный список пар (позиция в исходном списке кандидатов, кандидат). При этом позиция кандидата в полученном списке является его рейтингом (первый - лучший). Например, если исходный список кандидатов был [a, b, c], и самый похожий на исходный вопрос среди них - c, затем a, и в конце b, то функция должна вернуть список **[(2, c), (0, a), (1, b)]**.

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
from copy import deepcopy

In [ ]:
def rank_candidates(question, candidates, embeddings, tokenizer, dim=200):
    quest_vec = question_to_vec(question, embeddings, tokenizer, dim)

    # Преобразуем каждого кандидата в вектор и вычисляем сходство с вопросом
    cosine_similarities = []
    for candidate in candidates:
        candidate_vec = question_to_vec(candidate, embeddings, tokenizer, dim)
        # Используем только первую размерность векторов
        cosine_sim = cosine_similarity([quest_vec], [candidate_vec])
        cosine_similarities.append(cosine_sim)

    # Сортируем кандидатов по убыванию сходства и формируем ранжированный список
    sorted_candidates = sorted(zip(range(len(candidates)), candidates, cosine_similarities), key=lambda x: x[2], reverse=True)
    ranked_results = [(pos, cand) for pos, cand, _ in sorted_candidates]

    return ranked_results

# Пример использования
questions = 'Sending array via Ajax fails'
candidates = ['Getting all list items of an unordered list in PHP',      # второй эксперимент
               'WPF- How to update the changes in list item of a list',
               'select2 not displaying search results','srwyvs', 'Sending array via Ajax fail']
result = rank_candidates(questions, candidates, wv_embeddings, tokenizer)
print(result)

[(4, 'Sending array via Ajax fail'), (1, 'WPF- How to update the changes in list item of a list'), (0, 'Getting all list items of an unordered list in PHP'), (2, 'select2 not displaying search results'), (3, 'srwyvs')]


Протестируем работу функции на примерах ниже. Пусть $N=2$, то есть два эксперимента

Для первого экперимента можно полностью сравнить ваши ответы и правильные ответы. Но для второго эксперимента два ответа на кандидаты будут <b>скрыты</b>(*)

In [ ]:
def rank_candidates_for_multiple_questions(questions, all_candidates, embeddings, tokenizer, dim=200):
    # Список для хранения результатов для всех вопросов
    all_results = []

    # Применяем функцию rank_candidates для каждого вопроса и его кандидатов
    for question, candidates in zip(questions, all_candidates):
        results_for_question = rank_candidates(question, candidates, embeddings, tokenizer, dim)
        all_results.append(results_for_question)

    return all_results

questions = ['converting string to list', 'Sending array via Ajax fails']

candidates = [['Convert Google results object (pure js) to Python object', # первый эксперимент
               'C# create cookie from string and send it',
               'How to use jQuery AJAX for an outside domain?'],

              ['Getting all list items of an unordered list in PHP',      # второй эксперимент
               'WPF- How to update the changes in list item of a list',
               'select2 not displaying search results']]

results = rank_candidates_for_multiple_questions(questions, candidates, wv_embeddings, tokenizer)
print(results)

[[(1, 'C# create cookie from string and send it'), (0, 'Convert Google results object (pure js) to Python object'), (2, 'How to use jQuery AJAX for an outside domain?')], [(1, 'WPF- How to update the changes in list item of a list'), (0, 'Getting all list items of an unordered list in PHP'), (2, 'select2 not displaying search results')]]


Последовательность начальных индексов вы должны получить `для эксперимента 1`  1, 0, 2.

#### Ответ на вопрос 5 (какую последовательность начальных индексов получили для `эксперимента 2`(перечисление без запятой и пробелов, например, `102` для первого эксперимента)) :

**Последовательность для эксперимента 2 - 102**

Теперь мы можем оценить качество нашего метода. Запустим следующие два блока кода для получения результата. Вычисление расстояния между векторами занимает некоторое время (примерно 10 минут). Поэтому возьмем для validation 1000 примеров.

In [ ]:
from tqdm.notebook import tqdm

In [ ]:
validation_data[0][:5]

['How to print a binary heap tree without recursion?',
 'How do you best convert a recursive function to an iterative one?',
 'How can i use ng-model with directive in angular js',
 'flash: drawing and erasing',
 'toggle react component using hide show classname']

In [ ]:
validation_data[0][0]

'How to print a binary heap tree without recursion?'

In [ ]:
print ('len(validation_data)', len(validation_data))
print ('len(validation_data[0])', len(validation_data[0]))
print ('len(validation_data[0][0])', len(validation_data[0][0]))

len(validation_data) 3760
len(validation_data[0]) 1001
len(validation_data[0][0]) 50


In [ ]:
wv_ranking = []
max_validation_examples = 1000
for i, line in enumerate(tqdm(validation_data)):
    if i == max_validation_examples:
        break
    q, *ex = line
    # Вызывается функция rank_candidates, чтобы получить ранги кандидатов для данного вопроса q и списка кандидатов ex
    ranks = rank_candidates(q, ex, wv_embeddings, tokenizer)
    # print (ranks)

    # Извлекает ранги кандидатов из ranks и добавляет ранг кандидата с меткой '0' в список wv_ranking.
    wv_ranking.append([r[0] for r in ranks].index(0) + 1)

for k in tqdm([1, 5, 10, 100, 500, 1000]):
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k)))

  0%|          | 0/3760 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

DCG@   1: 0.285 | Hits@   1: 0.285
DCG@   5: 0.342 | Hits@   5: 0.393
DCG@  10: 0.360 | Hits@  10: 0.449
DCG@ 100: 0.406 | Hits@ 100: 0.679
DCG@ 500: 0.431 | Hits@ 500: 0.879
DCG@1000: 0.444 | Hits@1000: 1.000


In [ ]:
wv_ranking = []
max_validation_examples = 1000
for i, line in enumerate(tqdm(validation_data)):
    if i == max_validation_examples:
        break
    q, *ex = line
    # Вызывается функция rank_candidates, чтобы получить ранги кандидатов для данного вопроса q и списка кандидатов ex
    ranks = rank_candidates(q, ex, wv_embeddings, pymorphy2_tokenizer)
    # print (ranks)

    # Извлекает ранги кандидатов из ranks и добавляет ранг кандидата с меткой '0' в список wv_ranking.
    wv_ranking.append([r[0] for r in ranks].index(0) + 1)

for k in tqdm([1, 5, 10, 100, 500, 1000]):
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k)))

  0%|          | 0/3760 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

DCG@   1: 0.346 | Hits@   1: 0.346
DCG@   5: 0.432 | Hits@   5: 0.506
DCG@  10: 0.454 | Hits@  10: 0.575
DCG@ 100: 0.503 | Hits@ 100: 0.814
DCG@ 500: 0.522 | Hits@ 500: 0.960
DCG@1000: 0.526 | Hits@1000: 1.000


### Эмбеддинги, обученные на корпусе похожих вопросов

In [ ]:
train_data = read_corpus('/kaggle/input/trainvalid/train.tsv')

In [ ]:
print ('len(train_data)', len(train_data))
print ('len(train_data[0])', len(train_data[0]))
print ('len(train_data[0][0])', len(train_data[0][0]))

len(train_data) 1000000
len(train_data[0]) 2
len(train_data[0][0]) 25


In [ ]:
train_data[10]

['min value sql query', 'row with minimum value of a column\n']

Улучшим качество модели.<br>Склеим вопросы в пары и обучим на них модель Word2Vec из gensim. Выберем размер window.

In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
# стоп-слова, preproc
from nltk.corpus import stopwords
from gensim.models import Word2Vec

stopWords = set(stopwords.words('english'))
wnl = nltk.WordNetLemmatizer()
nltk.download('wordnet')

# def preproc_nltk(text):
#     return ' '.join([wnl.lemmatize(word) for word in word_tokenize(text.lower()) if word not in stopWords])

[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

#### Получение эмбеддингов без нормализации

In [ ]:
from nltk.tokenize import word_tokenize

def preproc_nltk(text):
    return ' '.join([word for word in word_tokenize(text.lower()) if word not in stopWords])
couple_list = [couple[0] + str(' ') + couple[1] for couple in train_data]

words = [preproc_nltk(question).split() for question in couple_list]

In [ ]:
parameter_table = pd.DataFrame(columns=['min_count', 'window',
                                      'DCG@1','DCG@5','DCG@10','DCG@100','DCG@500','DCG@1000',
                                     'Hits@1','Hits@5','Hits@10','Hits@100','Hits@500','Hits@1000'])
for min_count in range(1, 13, 2):
    for window in range(1, 13, 2):
        embeddings_trained = Word2Vec(words,
#                                           sg=0,
                                          vector_size=200,
                                          min_count=min_count,
                                          window=window,
                                          workers=4).wv
        wv_ranking = []
        max_validation_examples = 1000

        for i, line in enumerate(validation_data):
            if i == max_validation_examples:
                break
            q, *ex = line
            ranks = rank_candidates(q, ex, embeddings_trained, tokenizer)
            wv_ranking.append([r[0] for r in ranks].index(0) + 1)

        dcg_history = []
        hits_history = []

        for k in [1, 5, 10, 100, 500, 1000]:
            dcg_history.append(dcg_score(wv_ranking, k))
            hits_history.append(hits_count(wv_ranking, k))

        # Создаем DataFrame для текущей итерации
        data = pd.DataFrame({
                'min_count': [min_count],
                'window': [window],
                'DCG@1': [hits_history[0]],
                'DCG@5': [dcg_history[1]],
                'DCG@10': [dcg_history[2]],
                'DCG@100': [dcg_history[3]],
                'DCG@500': [dcg_history[4]],
                'DCG@1000': [dcg_history[5]],
                'Hits@1': [hits_history[0]],
                'Hits@5': [hits_history[1]],
                'Hits@10': [hits_history[2]],
                'Hits@100': [hits_history[3]],
                'Hits@500': [hits_history[4]],
                'Hits@1000': [hits_history[5]]
            })

        # Добавляем текущий DataFrame к основному DataFrame
        parameter_table = pd.concat([parameter_table, data], ignore_index=True)

        # Записываем DataFrame в файлы Excel и CSV
        parameter_table.to_excel('parameter_table.xlsx', index=False)
        parameter_table.to_csv('parameter_table.csv', index=False)

In [ ]:
# Таблица из предыдущего кода была сохранена и загружена в kaggle datasets, приведем ее:
file_path = '/kaggle/input/parameter-table/parameter_table.csv'

parameter_table = pd.read_csv(file_path)
display(parameter_table)

,min_count,window,DCG@1,DCG@5,DCG@10,DCG@100,DCG@500,DCG@1000,Hits@1,Hits@5,Hits@10,Hits@100,Hits@500,Hits@1000
0,1,1,0.258,0.321127,0.338958,0.380714,0.403489,0.422200,0.258,0.379,0.435,0.641,0.821,1.0
1,1,3,0.257,0.322122,0.344153,0.389998,0.413644,0.428078,0.257,0.380,0.449,0.677,0.862,1.0
2,1,5,0.266,0.327389,0.348823,0.395955,0.419387,0.433574,0.266,0.383,0.450,0.683,0.865,1.0
3,1,7,0.261,0.328747,0.348571,0.395268,0.418706,0.433014,0.261,0.392,0.454,0.683,0.864,1.0
4,1,9,0.263,0.332991,0.352657,0.397845,0.421229,0.435527,0.263,0.398,0.460,0.683,0.864,1.0
5,1,11,0.265,0.332369,0.354416,0.399082,0.422267,0.436647,0.265,0.393,0.462,0.683,0.863,1.0
6,3,1,0.260,0.325000,0.345315,0.388149,0.411846,0.428104,0.260,0.384,0.447,0.657,0.844,1.0
7,3,3,0.263,0.327984,0.349669,0.395584,0.420402,0.433340,0.263,0.389,0.456,0.683,0.877,1.0
8,3,5,0.266,0.331204,0.351686,0.399997,0.423705,0.437175,0.266,0.389,0.453,0.689,0.873,1.0
9,3,7,0.266,0.331607,0.354298,0.400305,0.423742,0.437392,0.266,0.391,0.462,0.689,0.871,1.0


In [ ]:
parameter_table.sort_values(by='DCG@5', ascending=False)

,min_count,window,DCG@1,DCG@5,DCG@10,DCG@100,DCG@500,DCG@1000,Hits@1,Hits@5,Hits@10,Hits@100,Hits@500,Hits@1000
23,7,11,0.277,0.341331,0.362819,0.408858,0.431646,0.445222,0.277,0.400,0.467,0.694,0.872,1.0
29,9,11,0.273,0.340562,0.362419,0.408832,0.431492,0.444757,0.273,0.401,0.469,0.698,0.875,1.0
17,5,11,0.274,0.339180,0.360391,0.406822,0.429810,0.443357,0.274,0.399,0.465,0.693,0.872,1.0
22,7,9,0.275,0.338863,0.362348,0.407198,0.430698,0.443953,0.275,0.397,0.470,0.691,0.875,1.0
34,11,9,0.268,0.337665,0.358858,0.405240,0.429525,0.442053,0.268,0.399,0.465,0.692,0.882,1.0
35,11,11,0.266,0.337372,0.358194,0.404306,0.427813,0.441205,0.266,0.400,0.465,0.691,0.874,1.0
28,9,9,0.269,0.337211,0.359090,0.405277,0.428731,0.441808,0.269,0.398,0.466,0.693,0.877,1.0
33,11,7,0.270,0.337110,0.357005,0.403700,0.428727,0.441164,0.270,0.398,0.460,0.688,0.883,1.0
27,9,7,0.271,0.337043,0.359872,0.404621,0.428839,0.441391,0.271,0.399,0.470,0.693,0.882,1.0
11,3,11,0.264,0.336978,0.357398,0.404349,0.426489,0.440219,0.264,0.401,0.465,0.697,0.870,1.0


In [ ]:
parameter_table[parameter_table['DCG@1000']==parameter_table['DCG@1000'].max()]

,min_count,window,DCG@1,DCG@5,DCG@10,DCG@100,DCG@500,DCG@1000,Hits@1,Hits@5,Hits@10,Hits@100,Hits@500,Hits@1000
23,7,11,0.277,0.341331,0.362819,0.408858,0.431646,0.445222,0.277,0.4,0.467,0.694,0.872,1.0


In [ ]:
#   обучим эмбеддинги без нормализации на оптимальных параметрах, добавив sg=1:

embeddings_trained = Word2Vec(words,
                              vector_size=200,
                              min_count=7,
                              window=11,
                              workers=4,
                              sg=1).wv

In [ ]:
tokenizers = [MyTokenizer(), Pymorphy2Tokenizer()]
count=0

for tokenizer in tokenizers:
    wv_ranking = []
    max_validation_examples = 1000
    for i, line in enumerate(validation_data):
        if i == max_validation_examples:
            break
        q, *ex = line
        ranks = rank_candidates(q, ex, embeddings_trained, tokenizer)
        wv_ranking.append([r[0] for r in ranks].index(0) + 1)
    if count < 2:
        print(str(tokenizer).split(".")[1].split(" ")[0])
    else:
        print(str(tokenizer).split("(")[0])
    for k in [1, 5, 10, 100, 500, 1000]:
        print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k)))
    print()
    count+=1

MyTokenizer
DCG@   1: 0.333 | Hits@   1: 0.333
DCG@   5: 0.400 | Hits@   5: 0.458
DCG@  10: 0.419 | Hits@  10: 0.518
DCG@ 100: 0.457 | Hits@ 100: 0.704
DCG@ 500: 0.477 | Hits@ 500: 0.857
DCG@1000: 0.492 | Hits@1000: 1.000

Pymorphy2Tokenizer
DCG@   1: 0.429 | Hits@   1: 0.429
DCG@   5: 0.524 | Hits@   5: 0.610
DCG@  10: 0.547 | Hits@  10: 0.683
DCG@ 100: 0.583 | Hits@ 100: 0.856
DCG@ 500: 0.595 | Hits@ 500: 0.952
DCG@1000: 0.600 | Hits@1000: 1.000



#### Получение эмбеддингов с нормализацией

In [ ]:
def preproc_nltk_norm(text):
    text = ' '.join([word for word in word_tokenize(text.lower()) if word not in stopWords])
    return Pymorphy2Tokenizer().tokenize(text)
words_norm = [preproc_nltk_norm(question) for question in couple_list]

In [ ]:
#   обучим эмбеддинги с нормализацией на оптимальных параметрах:

embeddings_trained_norm = Word2Vec(words_norm,
                            vector_size=200,
                            min_count=12,
                            window=10,
                            workers=4,
                            sg=1).wv

#### Качество ранжирования для 2 токенайзеров

In [ ]:
tokenizers = [MyTokenizer(),  Pymorphy2Tokenizer()]
count=0

for tokenizer in tokenizers:
    wv_ranking = []
    max_validation_examples = 1000
    for i, line in enumerate(validation_data):
        if i == max_validation_examples:
            break
        q, *ex = line
        ranks = rank_candidates(q, ex, embeddings_trained_norm, tokenizer)
        wv_ranking.append([r[0] for r in ranks].index(0) + 1)
    if count < 2:
        print(str(tokenizer).split(".")[1].split(" ")[0])
    else:
        print(str(tokenizer).split("(")[0])
    for k in [1, 5, 10, 100, 500, 1000]:
        print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k)))
    print()
    count+=1

MyTokenizer
DCG@   1: 0.331 | Hits@   1: 0.331
DCG@   5: 0.400 | Hits@   5: 0.463
DCG@  10: 0.419 | Hits@  10: 0.520
DCG@ 100: 0.456 | Hits@ 100: 0.702
DCG@ 500: 0.476 | Hits@ 500: 0.860
DCG@1000: 0.491 | Hits@1000: 1.000

Pymorphy2Tokenizer
DCG@   1: 0.429 | Hits@   1: 0.429
DCG@   5: 0.522 | Hits@   5: 0.605
DCG@  10: 0.545 | Hits@  10: 0.677
DCG@ 100: 0.581 | Hits@ 100: 0.852
DCG@ 500: 0.594 | Hits@ 500: 0.955
DCG@1000: 0.599 | Hits@1000: 1.000



## Вывод:

Токенизация pymorphy2_tokenizer (токенизатор возвращает слова, пунктуацию, применяет нормализацию) дает лучшее качество по сравнению с MyTokenize без нормализации. Spacy дает сопоставимое качество, но обрабатывает текст в несколько раз дольше.  


Нормализация слов в данном случае ухудшает метрики.


Лучшее качество показали эмбеддинги с min_count=7, window=11,c алгоритмом обучения sg=1 (Skip-Gram). Основная фича Skip-Gram-хорошее представление даже редких слов даже на небольшом наборе данных. Повышение min_count приводит к повышению  метрик DCG и Hits. Получение эмбеддингов с нормализацией приводит к ухудшению метрик. Вероятно, это связано с тем, что текст технический. Использование предобученных эмбеддингов дало немного более худшее качество по сравнению с обученными с подходом Word2Vec.


Качество невысокое как на предобученных эмбеддингах, так и на обученных. Вероятно, плохое качество получилось, тк использовался подход Word2Vec, который не является не является оптимальным к 2024 году. Например, для редких слов эмбеддинги получаются неоптимальными, а слова,имеющие один корень,обрабатываются по-разному.

В дальнейшем можно попробовать использовать GloVe/FastText